# Getting Financial Data - Pandas Datareader

## Introduction:
This time you will get data from a website.

In [78]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

import pandas_datareader.data as web

###  Create your time range (start and end variables). The start date should be 01/01/2015 and the end should today (whatever your today is).

In [79]:
date_start = datetime(2015, 1, 1)
date_end = datetime.date(datetime.now())

### Get an API key for one of the APIs that are supported by Pandas Datareader, preferably for AlphaVantage.

If you do not have an API key for any of the supported APIs, it is easiest to get one for AlphaVantage. (Note that the API key is shown directly after the signup. You do not receive it via e-mail.)

(For a full list of the APIs that are supported by Pandas Datareader, see here. As the APIs are provided by third parties, this list may change.)

### Use Pandas Datarader to read the daily time series for the Apple stock (ticker symbol AAPL) between 01/01/2015 and today, assign it to df_apple and print it.

[Examples](https://buildmedia.readthedocs.org/media/pdf/pandas-datareader/stable/pandas-datareader.pdf)

### Add a new column "stock" to the dataframe and add the ticker symbol

In [105]:
def read_stocks(symbol):
    df = web.DataReader(symbol,
                        "av-daily",
                        start = date_start, 
                        end = date_end, 
                        api_key = "Y9XW9A532PCH76U0")
    
    df["stock"] = symbol
    
    return df

In [106]:
df_apple = read_stocks("AAPL")
df_apple.head()

,open,high,low,close,volume,stock
2015-01-02,111.39,111.44,107.350,109.33,53204626,AAPL
2015-01-05,108.29,108.65,105.410,106.25,64285491,AAPL
2015-01-06,106.54,107.43,104.630,106.26,65797116,AAPL
2015-01-07,107.20,108.20,106.695,107.75,40105934,AAPL
2015-01-08,109.23,112.15,108.700,111.89,59364547,AAPL


### Repeat the two previous steps for a few other stocks, always creating a new dataframe: Tesla, IBM and Microsoft. (Ticker symbols TSLA, IBM and MSFT.)

In [107]:
df_tesla = read_stocks("TSLA")
df_ibm = read_stocks("IBM")
df_microsoft = read_stocks("MSFT")

### Combine the four separate dataFrames into one combined dataFrame df that holds the information for all four stocks

In [108]:
df = pd.concat([df_apple, df_tesla, df_ibm, df_microsoft])
df.head()

,open,high,low,close,volume,stock
2015-01-02,111.39,111.44,107.350,109.33,53204626,AAPL
2015-01-05,108.29,108.65,105.410,106.25,64285491,AAPL
2015-01-06,106.54,107.43,104.630,106.26,65797116,AAPL
2015-01-07,107.20,108.20,106.695,107.75,40105934,AAPL
2015-01-08,109.23,112.15,108.700,111.89,59364547,AAPL


### Shift the stock column into the index (making it a multi-level index consisting of the ticker symbol and the date).

In [109]:
df = df.reset_index()
df.rename(columns = {'index':'date'}, inplace = True)

df = df.set_index(['date', 'stock'])
df.head()

,,open,high,low,close,volume
date,stock,,,,,
2015-01-02,AAPL,111.39,111.44,107.350,109.33,53204626
2015-01-05,AAPL,108.29,108.65,105.410,106.25,64285491
2015-01-06,AAPL,106.54,107.43,104.630,106.26,65797116
2015-01-07,AAPL,107.20,108.20,106.695,107.75,40105934
2015-01-08,AAPL,109.23,112.15,108.700,111.89,59364547


### Create a dataFrame called vol, with the volume values.

In [166]:
vol = pd.DataFrame(df["volume"])
vol

,,volume
date,stock,
2015-01-02,AAPL,53204626
2015-01-05,AAPL,64285491
2015-01-06,AAPL,65797116
2015-01-07,AAPL,40105934
2015-01-08,AAPL,59364547
...,...,...
2022-04-25,MSFT,35678852
2022-04-26,MSFT,42047008
2022-04-27,MSFT,63477694


### Aggregate the data of volume to weekly.

In [167]:
vol = vol.reset_index()
vol.date = pd.to_datetime(vol.date)
vol

,date,stock,volume
0,2015-01-02,AAPL,53204626
1,2015-01-05,AAPL,64285491
2,2015-01-06,AAPL,65797116
3,2015-01-07,AAPL,40105934
4,2015-01-08,AAPL,59364547
...,...,...,...
7375,2022-04-25,MSFT,35678852
7376,2022-04-26,MSFT,42047008
7377,2022-04-27,MSFT,63477694
7378,2022-04-28,MSFT,33646570


In [168]:
vol = vol.pivot(index='date', columns='stock', values='volume')
vol

stock,AAPL,IBM,MSFT,TSLA
date,,,,
2015-01-02,53204626,5525341,27913852,4764443
2015-01-05,64285491,4880389,39673865,5368477
2015-01-06,65797116,6145670,36447854,6261936
2015-01-07,40105934,4701015,29114061,2968390
2015-01-08,59364547,4240585,29645202,3442509
...,...,...,...,...
2022-04-25,96046376,5748289,35678852,22780445
2022-04-26,94008394,4887395,42047008,44694524
2022-04-27,88063191,4369052,63477694,25652132


In [169]:
vol_weekly = vol.resample("W").sum()
vol_weekly

stock,AAPL,IBM,MSFT,TSLA
date,,,,
2015-01-04,53204626,5525341,27913852,4764443
2015-01-11,282868187,24440360,158596624,22622034
2015-01-18,304226647,23272056,157088136,30799137
2015-01-25,198737041,31230797,137352632,16215501
2015-02-01,465842684,32927307,437786778,15720217
...,...,...,...,...
2022-04-03,465395123,19183786,148667745,113080628
2022-04-10,390677922,16553021,143204193,128187848
2022-04-17,297460188,15342641,115664768,79625639


### Find all the volume traded in the year of 2015

In [176]:
vol_2015 = vol.resample("Y").sum().loc["2015-12-31"]
vol_2015

stock
AAPL    13064316775
IBM      1105545521
MSFT     9057582311
TSLA     1086708380
Name: 2015-12-31 00:00:00, dtype: int64